# Подключаемся к БД

In [3]:
def query_sql(query, is_insert=False):
    conn = ps.connect(host="ru.tuna.am", 
                      port = 35663, 
                      database="dev", 
                      user=os.getenv("POSTGRES_USER"), 
                      password=os.getenv("POSTGRES_PASSWORD"))
    cursor = conn.cursor()
    cursor.execute(query)
    if is_insert == False:
        df = cursor.fetchall()
        field_names = [i[0].lower() for i in cursor.description]
        df = pd.DataFrame(df, columns=field_names)
        df.columns = field_names
    else:
        df = pd.DataFrame()
    cursor.execute("DEALLOCATE ALL")
    cursor.close()
    return df

# 1. Извлечение информации при помощи SQL-запросов

In [2]:
query_sql(f'''

SELECT * 
FROM {schema}.employees

''')

NameError: name 'schema' is not defined

# 2. Фильтрация строк **WHERE**, **LIKE**, **ILIKE**, **AND**, **OR**

**WHERE**

In [2]:
query_sql(f'''

SELECT * 
FROM {schema}.employees 
WHERE department = 'IT'

''')




NameError: name 'schema' is not defined

**LIKE/ILIKE**

In [ ]:
query_sql(f'''

SELECT * 
FROM {schema}.employees 
WHERE email LIKE '%@example_com'

''')

,id,name,department,salary,hire_date,email
0,1,Иван Иванов,HR,50000.00,2020-01-15,ivanov@example.com
1,2,Петр Петров,IT,None,2019-03-10,petrov@example.com
2,3,Светлана Смирнова,Finance,60000.00,2021-07-22,smit@example.com
3,4,Алексей Кузнецов,IT,80000.00,2018-11-05,kuznecov@example.com
4,5,Мария Иванова,HR,52000.00,2022-02-17,maria@example.com


**AND**

In [ ]:
query_sql(f'''

SELECT * 
FROM {schema}.employees 
WHERE department = 'HR' 
    AND salary > 51000
    
''')

,id,name,department,salary,hire_date,email
0,5,Мария Иванова,HR,52000.00,2022-02-17,maria@example.com


**OR**

In [ ]:
query_sql(f'''

SELECT * 
FROM {schema}.employees 
WHERE department = 'HR' 
    OR salary > 75000

''')

,id,name,department,salary,hire_date,email
0,1,Иван Иванов,HR,50000.00,2020-01-15,ivanov@example.com
1,4,Алексей Кузнецов,IT,80000.00,2018-11-05,kuznecov@example.com
2,5,Мария Иванова,HR,52000.00,2022-02-17,maria@example.com


# 3. Сортировка строк: **ORDER BY**, **LIMIT**, **OFFSET**

**ORDER BY**

In [ ]:
query_sql(f'''

SELECT * 
FROM {schema}.employees 
ORDER BY salary DESC

''')

,id,name,department,salary,hire_date,email
0,2,Петр Петров,IT,None,2019-03-10,petrov@example.com
1,4,Алексей Кузнецов,IT,80000.00,2018-11-05,kuznecov@example.com
2,3,Светлана Смирнова,Finance,60000.00,2021-07-22,smit@example.com
3,5,Мария Иванова,HR,52000.00,2022-02-17,maria@example.com
4,1,Иван Иванов,HR,50000.00,2020-01-15,ivanov@example.com


**ORDER BY**

In [ ]:
query_sql(f'''

SELECT * 
FROM {schema}.employees 
ORDER BY hire_date 
LIMIT 3

''')

,id,name,department,salary,hire_date,email
0,4,Алексей Кузнецов,IT,80000.00,2018-11-05,kuznecov@example.com
1,2,Петр Петров,IT,None,2019-03-10,petrov@example.com
2,1,Иван Иванов,HR,50000.00,2020-01-15,ivanov@example.com


**ORDER BY ... OFFSET ...**

In [ ]:
query_sql(f'''

SELECT * 
FROM {schema}.employees 
ORDER BY salary 
--LIMIT 2 
OFFSET 2

''')

,id,name,department,salary,hire_date,email
0,3,Светлана Смирнова,Finance,60000.00,2021-07-22,smit@example.com
1,4,Алексей Кузнецов,IT,80000.00,2018-11-05,kuznecov@example.com
2,2,Петр Петров,IT,None,2019-03-10,petrov@example.com


# 4. Условные выражения: **CASE**, **COALESCE**, **NULLIF**, **GREATEST**, **LEAST**

**CASE**

In [ ]:
query_sql(f'''

    SELECT name,
           salary,
           CASE WHEN salary > 60000 THEN 'Высокая'
                WHEN salary BETWEEN 50000 AND 60000 THEN 'Средняя'
                ELSE 'Низкая' 
            END AS salary_level
    FROM {schema}.employees

''')

,name,salary,salary_level
0,Иван Иванов,50000.00,Средняя
1,Петр Петров,None,Низкая
2,Светлана Смирнова,60000.00,Средняя
3,Алексей Кузнецов,80000.00,Высокая
4,Мария Иванова,52000.00,Средняя


**COALESCE**

In [ ]:
query_sql(f'''

    SELECT 
        name, 
        COALESCE(email, 'Нет email') AS email_info 
    FROM {schema}.employees

''')

,name,email_info
0,Иван Иванов,ivanov@example.com
1,Петр Петров,petrov@example.com
2,Светлана Смирнова,smit@example.com
3,Алексей Кузнецов,kuznecov@example.com
4,Мария Иванова,maria@example.com


**NULLIF**

In [ ]:
query_sql(f'''

    SELECT 
        name, 
        NULLIF(department, 'HR') AS dept_without_hr 
    FROM {schema}.employees

''')

,name,dept_without_hr
0,Иван Иванов,None
1,Петр Петров,IT
2,Светлана Смирнова,Finance
3,Алексей Кузнецов,IT
4,Мария Иванова,None


**GREATEST / LEAST**

In [ ]:
query_sql(f'''

    SELECT 
        name, 
        salary,
        GREATEST(salary) AS max_salary, 
        LEAST(salary, 4000) AS min_salary 
    FROM {schema}.employees

''')

,name,salary,max_salary,min_salary
0,Иван Иванов,50000.00,50000.00,4000
1,Петр Петров,None,None,4000
2,Светлана Смирнова,60000.00,60000.00,4000
3,Алексей Кузнецов,80000.00,80000.00,4000
4,Мария Иванова,52000.00,52000.00,4000


# 5. Работа со строками: **SUBSTRING**, **LOWER**, **UPPER**, **TRIM**, **LENGTH**

In [ ]:
query_sql(f'''

    SELECT name, 
           SUBSTRING(name, 1, 5) AS name_substring,
           LOWER(email) AS email_lower,
           UPPER(name) AS name_upper,
           LENGTH(name) AS name_length,
           TRIM('   текст с пробелами   ') AS trimmed_text
    FROM {schema}.employees
    
''')

,name,name_substring,email_lower,name_upper,name_length,trimmed_text
0,Иван Иванов,Иван,ivanov@example.com,ИВАН ИВАНОВ,11,текст с пробелами
1,Петр Петров,Петр,petrov@example.com,ПЕТР ПЕТРОВ,11,текст с пробелами
2,Светлана Смирнова,Светл,smit@example.com,СВЕТЛАНА СМИРНОВА,17,текст с пробелами
3,Алексей Кузнецов,Алекс,kuznecov@example.com,АЛЕКСЕЙ КУЗНЕЦОВ,16,текст с пробелами
4,Мария Иванова,Мария,maria@example.com,МАРИЯ ИВАНОВА,13,текст с пробелами


# 6. Работа с датами

In [ ]:
query_sql(f'''

    SELECT name, 
           hire_date,
           DATE(hire_date) AS hire_date_only,
           TO_CHAR(hire_date, 'yyyy-mm'),
           DATE_TRUNC('month', hire_date),
           hire_date + 1,
           hire_date + interval '1 hour',
           DATE_PART('DOW', hire_date)
    FROM {schema}.employees

''')

,name,hire_date,hire_date_only,to_char,date_trunc,?column?,?column?,date_part
0,Иван Иванов,2020-01-15,2020-01-15,2020-01,2020-01-01 00:00:00+00:00,2020-01-16,2020-01-15 01:00:00,3.0
1,Петр Петров,2019-03-10,2019-03-10,2019-03,2019-03-01 00:00:00+00:00,2019-03-11,2019-03-10 01:00:00,0.0
2,Светлана Смирнова,2021-07-22,2021-07-22,2021-07,2021-07-01 00:00:00+00:00,2021-07-23,2021-07-22 01:00:00,4.0
3,Алексей Кузнецов,2018-11-05,2018-11-05,2018-11,2018-11-01 00:00:00+00:00,2018-11-06,2018-11-05 01:00:00,1.0
4,Мария Иванова,2022-02-17,2022-02-17,2022-02,2022-02-01 00:00:00+00:00,2022-02-18,2022-02-17 01:00:00,4.0


# 7. Агрегатные функции: **COUNT**, **SUM**, **MIN**, **MAX**, **AVG**, **GROUP BY**, **HAVING**

In [ ]:
query_sql(f'''

    SELECT 
        department, 
        COUNT(*) AS count_employees, 
        COUNT(salary) as count_salary,
        AVG(salary) AS avg_salary_1,
        SUM(salary) / COUNT(*) AS avg_salary_2,
        SUM(salary) / COUNT(salary) AS avg_salary_3,
        MIN(salary) as min_salary,
        MAX(salary) as max_salary
    FROM {schema}.employees
    GROUP BY department
    
''')

,department,count_employees,count_salary,avg_salary_1,avg_salary_2,avg_salary_3,min_salary,max_salary
0,Finance,1,1,60000.000000000000,60000.000000000000,60000.000000000000,60000.00,60000.00
1,IT,2,1,80000.000000000000,40000.000000000000,80000.000000000000,80000.00,80000.00
2,HR,2,2,51000.000000000000,51000.000000000000,51000.000000000000,50000.00,52000.00


In [ ]:
query_sql(f'''

    SELECT 
        department, 
        COUNT(*) AS count_employees
    FROM {schema}.employees
    GROUP BY department
    HAVING COUNT(*) > 1

''')

,department,count_employees
0,IT,2
1,HR,2


# 8. Оконные функции: **row_number**, **rank**, **dense_rank**, **lag**, **lead**

In [ ]:
query_sql(f'''

    SELECT 
        name, 
        salary,
        department,
        ROW_NUMBER() OVER (ORDER BY salary DESC) AS row_num,
        RANK() OVER (ORDER BY department DESC) AS rank,
        DENSE_RANK() OVER (ORDER BY department DESC) AS dense_rank,
        LAG(salary, 1) OVER (ORDER BY salary DESC) AS prev_salary,
        LEAD(salary, 1) OVER (ORDER BY salary DESC) AS next_salary,
        SUM(salary) OVER() as sal_all,
        SUM(salary) OVER(order by salary DESC) as sal_all
    FROM {schema}.employees
    order by salary DESC

''')

,name,salary,department,row_num,rank,dense_rank,prev_salary,next_salary,sal_all,sal_all
0,Петр Петров,None,IT,1,1,1,None,80000.00,242000.00,None
1,Алексей Кузнецов,80000.00,IT,2,1,1,None,60000.00,242000.00,80000.00
2,Светлана Смирнова,60000.00,Finance,3,5,3,80000.00,52000.00,242000.00,140000.00
3,Мария Иванова,52000.00,HR,4,3,2,60000.00,50000.00,242000.00,192000.00
4,Иван Иванов,50000.00,HR,5,3,2,52000.00,None,242000.00,242000.00


# 9. Типы связей в БД и оператор **JOIN**

In [ ]:

query_sql(f'''

    SELECT e.name, d.dept_name, d.location
    FROM {schema}.employees e
        JOIN {schema}.departments d 
            ON e.department = d.dept_name

''')

,name,dept_name,location
0,Мария Иванова,HR,Москва
1,Иван Иванов,HR,Москва
2,Алексей Кузнецов,IT,Санкт-Петербург
3,Петр Петров,IT,Санкт-Петербург
4,Светлана Смирнова,Finance,Москва


In [ ]:

query_sql(f'''

    SELECT e.name, d.dept_name, d.location
    FROM {schema}.employees e
        RIGHT JOIN {schema}.departments d 
            ON e.department = d.dept_name

''')

,name,dept_name,location
0,Мария Иванова,HR,Москва
1,Иван Иванов,HR,Москва
2,Алексей Кузнецов,IT,Санкт-Петербург
3,Петр Петров,IT,Санкт-Петербург
4,Светлана Смирнова,Finance,Москва
5,None,DE,Краснодар


In [ ]:
query_sql(f'''

    SELECT e.name, d.dept_name, d.location
    FROM {schema}.employees e
        CROSS JOIN {schema}.departments d 

''')

,name,dept_name,location
0,Иван Иванов,HR,Москва
1,Петр Петров,HR,Москва
2,Светлана Смирнова,HR,Москва
3,Алексей Кузнецов,HR,Москва
4,Мария Иванова,HR,Москва
5,Иван Иванов,IT,Санкт-Петербург
6,Петр Петров,IT,Санкт-Петербург
7,Светлана Смирнова,IT,Санкт-Петербург
8,Алексей Кузнецов,IT,Санкт-Петербург
9,Мария Иванова,IT,Санкт-Петербург


# 11. Подзапросы

In [ ]:
query_sql(f'''

    SELECT name, salary
    FROM {schema}.employees
    WHERE salary > (SELECT AVG(salary) 
                    FROM {schema}.employees)

''')

,name,salary
0,Алексей Кузнецов,80000.00


In [ ]:
query_sql(f'''
    WITH t1 AS (
        SELECT AVG(salary) as avg_salary FROM {schema}.employees
    )
    SELECT name, salary
    FROM {schema}.employees
    WHERE salary > (SELECT avg_salary FROM t1)

''')

,name,salary
0,Алексей Кузнецов,80000.00
